In [3]:
#User Items
#Importo Librerias
import pandas as pd
import gzip

PlayTimeGenre

In [4]:
# Asigno al DataFrame el archivo csv df_sg.csv
#df_sg=pd.read_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_sg.csv')
df_sg=pd.read_csv('./Dataset/df_sg.csv')

In [118]:
# Asigno al DataFrame el archivo csv df_ui.csv.gz
#df_ui = pd.read_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_ui.csv.gz', compression='gzip')
df_ui = pd.read_csv('./Dataset/df_ui.csv.gz', compression='gzip')

In [119]:
# Crear dataframe base de PlayTimeGenre
df_sgui_pg=pd.merge(df_sg[['item_id','genres','release_date']],df_ui[['item_id','playtime_forever']],on='item_id',how='left')
df_sgui_pg.head()

,item_id,genres,release_date,playtime_forever
0,761140,Action,2018,NaN
1,761140,Casual,2018,NaN
2,761140,Indie,2018,NaN
3,761140,Simulation,2018,NaN
4,761140,Strategy,2018,NaN


In [5]:
# Elimino nulos
df_sgui_pg.dropna(inplace=True)

In [6]:
# Creo dataframe final para PlayTimeGenre
df_PlayTimeGenre = df_sgui_pg.groupby(['genres', 'release_date'])['playtime_forever'].sum().reset_index()
df_PlayTimeGenre.head()

,genres,release_date,playtime_forever
0,Action,1983,3582.0
1,Action,1984,384.0
2,Action,1988,16243.0
3,Action,1989,607.0
4,Action,1990,18787.0


In [7]:
# Guardo dataframe en PlayTimeGenre.csv
#df_PlayTimeGenre.to_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_PlayTimeGenre.csv',index=False)
df_PlayTimeGenre.to_csv('./Dataset/df_PlayTimeGenre.csv',index=False)

In [8]:
#Funcion PlayTimeGenre
genre='RPG'
#df_playtimegenre= pd.read_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_PlayTimeGenre.csv')
df_playtimegenre= pd.read_csv('./Dataset/df_PlayTimeGenre.csv')
df_filtered_by_genre = df_playtimegenre[df_playtimegenre['genres'] == genre]
gender_hours = df_filtered_by_genre.groupby('release_date')['playtime_forever'].sum()
max_time_play_gender = gender_hours.idxmax()
result = {
  "Año de lanzamiento con más horas jugadas para Género {}".format(genre) : max_time_play_gender
  }
result

{'Año de lanzamiento con más horas jugadas para Género RPG': 2011}

UserForGenre

In [14]:
# Creo Lista de Generos
df_generos_diferentes = df_sg['genres'].unique()
df_generos_diferentes.sort()

In [77]:
# Relaciono dataframes steam_games con user_items
df_sgui_gu=pd.merge(df_sg[['item_id','genres','release_date']],df_ui[['item_id','playtime_forever','user_id']],on='item_id',how='left')
df_sgui_gu.dropna(inplace=True)

# Creo lista de generos
usuario_por_genero = []
df_UserForGenre = pd.DataFrame(columns=['user_id', 'genres', 'release_date', 'playtime_forever'])


for genero in df_generos_diferentes:
    # Filtramos las filas del género actual
    df_genero = df_sgui_gu[df_sgui_gu['genres'] == genero]
    # Filtramos el usuario del género actual
    if not df_genero.empty:
        # Filtramos al usuario con mayor 'playtime_forever' del género actual
        df_usuario = df_genero.groupby('user_id')['playtime_forever'].sum().div(60).idxmax()
        # Agrupamos los datos por usuario, genero y año y calculamos la suma de 'playtime_forever'
        df_usuario_genero_año = df_genero[df_genero['user_id'] == df_usuario].groupby(['user_id','genres', 'release_date'])['playtime_forever'].sum().div(60).reset_index()
        # Agregamos el df_usuario_genero_año a la lista usuario_por_genero
        usuario_por_genero.append(df_usuario_genero_año)
    else:
        pass
# Convertimos los resultados en un DataFrame
for df in usuario_por_genero:
    # Agregamos al DataFrame df_UserForGenre los dataframes de cada genero y usuario
    df_UserForGenre = pd.concat([df_UserForGenre, df], ignore_index=True)
df_UserForGenre['playtime_forever'] = df_UserForGenre['playtime_forever'].astype('int')
df_UserForGenre.drop(df_UserForGenre[df_UserForGenre['playtime_forever'] == 0].index, inplace=True)
df_UserForGenre = df_UserForGenre.reset_index(drop=True)
df_UserForGenre.head()

,user_id,genres,release_date,playtime_forever
0,Sp3ctre,Action,1995,3
1,Sp3ctre,Action,2000,1177
2,Sp3ctre,Action,2002,3
3,Sp3ctre,Action,2003,127
4,Sp3ctre,Action,2004,2123


In [47]:
# Guardo Dataframe en df_UserForGenre.csv
# df_UserForGenre.to_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_UserForGenre.csv',index=False)
df_UserForGenre.to_csv('./Dataset/df_UserForGenre.csv',index=False)

In [48]:
#Funcion UserForGenre
genre='Action'
# df_userforgenre= pd.read_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_UserForGenre.csv')
df_userforgenre= pd.read_csv('./Dataset/df_UserForGenre.csv')
df_filtered_by_genre = df_userforgenre[df_userforgenre['genres']==genre]

user_hours = df_filtered_by_genre.groupby('user_id')['playtime_forever'].sum()
max_time_play_user = user_hours.idxmax()

max_user_df = df_filtered_by_genre[df_filtered_by_genre['user_id'] == max_time_play_user]
hours_by_year = max_user_df.groupby('release_date')['playtime_forever'].sum().reset_index()

hours_years_list = [
    {'Año':year, 'Horas': hours} for year, hours in zip(hours_by_year['release_date'],hours_by_year['playtime_forever'])
                    ]
result = {
  'Usuario con más horas jugadas para genero {}'.format(genre):max_time_play_user,
  'Horas jugadas': hours_years_list
  }
result

{'Usuario con más horas jugadas para genero Action': 'Sp3ctre',
 'Horas jugadas': [{'Año': 1995, 'Horas': 3},
  {'Año': 2000, 'Horas': 1177},
  {'Año': 2002, 'Horas': 3},
  {'Año': 2003, 'Horas': 127},
  {'Año': 2004, 'Horas': 2123},
  {'Año': 2005, 'Horas': 355},
  {'Año': 2006, 'Horas': 14},
  {'Año': 2007, 'Horas': 1879},
  {'Año': 2008, 'Horas': 3},
  {'Año': 2009, 'Horas': 1805},
  {'Año': 2010, 'Horas': 1301},
  {'Año': 2011, 'Horas': 2581},
  {'Año': 2012, 'Horas': 6304},
  {'Año': 2013, 'Horas': 2007},
  {'Año': 2014, 'Horas': 2178},
  {'Año': 2015, 'Horas': 5125},
  {'Año': 2016, 'Horas': 492},
  {'Año': 2017, 'Horas': 722}]}

UsersRecommend

In [157]:
# Asigno al DataFrame el archivo csv df_ur.csv
#df_ur=pd.read_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_ur.csv')
df_ur=pd.read_csv('./Dataset/df_ur.csv')
# Filtro el dataframe por recomendado positiva o neutral 
df_ur_r = df_ur[
    (df_ur['recommend'] == True) &
    (df_ur['sentiment_analysis'] != 0)] 

df_sg_r = df_sg[['item_id', 'app_name','release_date']].drop_duplicates().reset_index(drop=True)
#Merge de los dataframes de steam_games filtrados y user_reviews filtrados
df_sgur_r=pd.merge(df_sg_r[['item_id', 'app_name','release_date']],df_ur_r[['item_id','recommend','sentiment_analysis']],on='item_id',how='inner')

# Creo Lista de Años
df_años_diferentes = df_sg_r['release_date'].unique()
df_años_diferentes.sort()
#df_años_diferentes

# Creo lista de años
app_por_año = []
df_UsersRecommend = pd.DataFrame(columns=['app_name', 'Count', 'release_date'])

for año in df_años_diferentes:
    # Filtramos las filas del género actual
    df_año = df_sgur_r[df_sgur_r['release_date'] == año]
    # Filtramos el usuario del género actual
    if not df_año.empty:
        top_games_año = df_año['app_name'].value_counts().head(3)
        
        # Crea un DataFrame de la serie
        top_games_año_df = top_games_año.reset_index()
        # Renombra la columna
        top_games_año_df.columns = ['app_name', 'Count']
        # Añade una nueva columna con el valor del año
        top_games_año_df['release_date'] = df_año['release_date'].iloc[0]  

        app_por_año.append(top_games_año_df)
    else:
        pass
# Convertimos los resultados en un DataFrame
for df in app_por_año:
    # Agregamos al DataFrame df_UserForGenre los dataframes de cada genero y usuario
    df_UsersRecommend = pd.concat([df_UsersRecommend, df], ignore_index=True)

df_UsersRecommend.head()

,app_name,Count,release_date
0,Sword of the Samurai,1,1989
1,Commander Keen,3,1990
2,LOOM™,1,1990
3,Crystal Caves,1,1991
4,Putt-Putt® Joins the Parade,1,1992


In [158]:
# Guardo Dataframe en df_UsersRecommend.csv
# df_UsersRecommend.to_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_UsersRecommend.csv',index=False)
df_UsersRecommend.to_csv('./Dataset/df_UsersRecommend.csv',index=False)

In [164]:
#Funcion UsersRecommend
year=1990
# df_userforgenre= pd.read_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_UserForGenre.csv')
df_usersrecommend= pd.read_csv('./Dataset/df_UsersRecommend.csv')

df_filtered_by_year = df_usersrecommend[df_usersrecommend['release_date']==year]

top_games = df_filtered_by_year['app_name'].value_counts().head(3)

#Convertir el resultado a un formato de lista de diccionarios
result = [{"Puesto numero {} más recomendado es".format(i+1): juego} for i, juego in enumerate(top_games.index)]
result



[{'Puesto numero 1 más recomendado es': 'Commander Keen'},
 {'Puesto numero 2 más recomendado es': 'LOOM™'}]

UsersNotRecommend

In [159]:
# Asigno al DataFrame el archivo csv df_ur.csv
#df_ur=pd.read_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_ur.csv')
df_ur=pd.read_csv('./Dataset/df_ur.csv')
# Filtro el dataframe por no recomendado negativa 
df_ur_nr = df_ur[
    (df_ur['recommend'] == False) &
    (df_ur['sentiment_analysis'] == 0)] 

df_sg_nr = df_sg[['item_id', 'app_name','release_date']].drop_duplicates().reset_index(drop=True)
#Merge de los dataframes de steam_games filtrados y user_reviews filtrados
df_sgur_nr=pd.merge(df_sg_nr[['item_id', 'app_name','release_date']],df_ur_nr[['item_id','recommend','sentiment_analysis']],on='item_id',how='inner')

# Creo Lista de Años
df_años_diferentes = df_sg_nr['release_date'].unique()
df_años_diferentes.sort()
#df_años_diferentes

# Creo lista de años
app_por_año = []
df_UsersNotRecommend = pd.DataFrame(columns=['app_name', 'Count', 'release_date'])

for año in df_años_diferentes:
    # Filtramos las filas del género actual
    df_año = df_sgur_nr[df_sgur_nr['release_date'] == año]
    # Filtramos el usuario del género actual
    if not df_año.empty:
        top_games_año = df_año['app_name'].value_counts().head(3)
        
        # Crea un DataFrame de la serie
        top_games_año_df = top_games_año.reset_index()
        # Renombra la columna
        top_games_año_df.columns = ['app_name', 'Count']
        # Añade una nueva columna con el valor del año
        top_games_año_df['release_date'] = df_año['release_date'].iloc[0]  

        app_por_año.append(top_games_año_df)
    else:
        pass
# Convertimos los resultados en un DataFrame
for df in app_por_año:
    # Agregamos al DataFrame df_UserForGenre los dataframes de cada genero y usuario
    df_UsersNotRecommend = pd.concat([df_UsersNotRecommend, df], ignore_index=True)

df_UsersNotRecommend.head()

,app_name,Count,release_date
0,Commander Keen,1,1990
1,Team Fortress Classic,3,1999
2,Sven Co-op,2,1999
3,Half-Life: Opposing Force,1,1999
4,Deus Ex: Game of the Year Edition,2,2000


In [160]:
# Guardo Dataframe en df_UsersRecommend.csv
# df_UsersRecommend.to_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_UsersRecommend.csv',index=False)
df_UsersNotRecommend.to_csv('./Dataset/df_UsersNotRecommend.csv',index=False)

In [165]:
#Funcion UsersNotRecommend
year=1990
# df_userforgenre= pd.read_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_UserForGenre.csv')
df_usersnotrecommend= pd.read_csv('./Dataset/df_UsersRecommend.csv')

df_filtered_by_year = df_usersnotrecommend[df_usersnotrecommend['release_date']==year]

top_games = df_filtered_by_year['app_name'].value_counts().head(3)

#Convertir el resultado a un formato de lista de diccionarios
result = [{"Puesto numero {} menos recomendado es".format(i+1): juego} for i, juego in enumerate(top_games.index)]
result

[{'Puesto numero 1 menos recomendado es': 'Commander Keen'},
 {'Puesto numero 2 menos recomendado es': 'LOOM™'}]

sentiment_analysis

In [28]:
df_sentiment_analysis_names=[['Codigo','sentiment_analysis'],['0','Negativo'],['1','Neutro'],['2','Positivo']]
df_sentiment_analysis_names
headers = df_sentiment_analysis_names.pop(0)

# Create the DataFrame with the remaining lists as data and using the headers
df_sentiment_analysis_codes = pd.DataFrame(df_sentiment_analysis_names, columns=headers)

# Display the DataFrame
print(df_sentiment_analysis_codes)

  Codigo sentiment_analysis
0      0           Negativo
1      1             Neutro
2      2           Positivo


In [25]:
# Asigno al DataFrame el archivo csv df_ur.csv
#df_ur=pd.read_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_ur.csv')
df_ur=pd.read_csv('./Dataset/df_ur.csv')

#Elimino Duplicados del Dataframe
df_sg_sa = df_sg[['item_id', 'app_name','release_date']].drop_duplicates().reset_index(drop=True)

#Merge de los dataframes de steam_games filtrados y user_reviews
df_sgur_sa=pd.merge(df_sg_sa[['item_id', 'app_name','release_date']],df_ur[['item_id','sentiment_analysis']],on='item_id',how='inner')

# Creo Lista de Años
df_años_diferentes = df_sgur_sa['release_date'].unique()
df_años_diferentes.sort()
#df_años_diferentes

# Creo lista de años
sa_por_año = []
df_sentiment_analysis = pd.DataFrame(columns=['release_date', 'sentiment_analysis', 'Count'])

for año in df_años_diferentes:
    # Filtramos las filas del género actual
    df_año = df_sgur_sa[df_sgur_sa['release_date'] == año]
    # Filtramos el usuario del género actual
    if not df_año.empty:
        
        #Filtra por total de tipo de sentiment_analysis
        df_sentiment_analysis_a = df_año['sentiment_analysis'].value_counts()
        #int(df_sentiment_analysis_a.get(0, 0))
        #print(df_año['sentiment_analysis'].value_counts())
        #df_año.head(60)
        #print(año)
        #print(df_sentiment_analysis_a.info())
        # Crea un DataFrame de la serie
        top_sa_año_df = df_sentiment_analysis_a.reset_index()
        # Renombra la columna
        top_sa_año_df.columns = ['sentiment_analysis', 'Count']
        # Añade una nueva columna con el valor del año
        top_sa_año_df['release_date'] = df_año['release_date'].iloc[0]  

        sa_por_año.append(top_sa_año_df)
    else:
        pass
# Convertimos los resultados en un DataFrame
for df in sa_por_año:
    # Agregamos al DataFrame df_sentiment_analysis los dataframes de cada genero y usuario
    df_sentiment_analysis = pd.concat([df_sentiment_analysis, df], ignore_index=True)

df_sentiment_analysis.head(60)
#print('Negativo: '.counts_by_sentiment.get(0, 0))#,
#'Neutro': counts_by_sentiment.get(1, 0),
#'Positivo': counts_by_sentiment.get(2, 0)}

AttributeError: 'str' object has no attribute 'counts_by_sentiment'

In [20]:
# Guardo Dataframe en df_UsersRecommend.csv
# df_UsersRecommend.to_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_UsersRecommend.csv',index=False)
df_sentiment_analysis.to_csv('./Dataset/df_sentiment_analysis.csv',index=False)